In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
from os import walk
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
import pickle

Loading the dataset and normalizing

In [ ]:
# Choose location for SCADA and Acceleration data
loc = 'turbine' # location
modelID = 1
# (0 for SCADA only, 1 for SCADA+Acc17, 2 for SCADA+Acc38, 3 for SCADA+Acc77, 
# 4 for SCADA+Acc17&38, 5 for SCADA+Acc17&38&77 
include_wave = 'yes'
duration = '24M'

durations = ['3M','6M','9M','12M','15M','18M','21M','24M']
train_end_dates = ['2018-03-31 23:50:00+00:00', '2018-06-30 23:50:00+00:00', '2018-09-30 23:50:00+00:00', 
                  '2018-12-31 23:50:00+00:00', '2019-03-31 23:50:00+00:00', '2019-06-30 23:50:00+00:00',
                  '2019-09-30 23:50:00+00:00', '2019-12-31 23:50:00+00:00']
train_end_date = train_end_dates[durations.index(duration)]

# Laod train data 
train_input = pd.read_pickle('DATA/train_input')
train_output = pd.read_pickle('DATA/train_output')
index = train_input.columns

train_input = train_input.loc['2018-01-01 00:00:00+00:00':train_end_date]
train_output= train_output.loc['2018-01-01 00:00:00+00:00':train_end_date]
# print(train_input.shape)

# Normlaization of input data
# Data normalization according to training dataset/ model
filehandler = open('Weights/Norm', 'rb') 
std_scaler = pickle.load(filehandler)
inputn = pd.DataFrame(std_scaler.transform(train_input), columns=train_input.columns) 
# inputn is still a daraframe with numeric index
outputn = train_output/10**6  #  change of units, outputn is still a daraframe with time index

Retrive features based on the modelID

In [ ]:
index1 = pd.core.indexes.base.Index([]) # create a blank index array
if include_wave == 'yes': 
    index1 = index1.append(index[0:3])
if modelID == 1: # Acc17
    index1 = index1.append(index[[3,4,9,10,15,16]])
if modelID == 2: # Acc38
    index1 = index1.append(index[[5,6,11,12,17,18]])
if modelID == 3: # Acc77
    index1 = index1.append(index[[7,8,13,14,19,20]])
if modelID == 4: # Acc17&38
    index1 = index1.append(index[[3,4,5,6,9,10,11,12,15,16,17,18]])   
if modelID == 5: # Acc17&38&77
    index1 = index1.append(index[3:21])

index1 = index1.append(index[21:]) # SCADA
X = inputn[index1].values
Y = outputn.values
print(X.shape)
print(Y.shape)

NN architecture and compiling

In [ ]:
in_dim = X.shape[1]
out_dim = Y.shape[1]
VSnet = Sequential()
VSnet.add(Dense(64, input_dim=in_dim, activation="relu"))
VSnet.add(Dense(128, activation="relu"))
VSnet.add(Dense(64, activation="relu"))
VSnet.add(Dense(out_dim, activation="relu"))

loss = losses.mean_absolute_error
optimizer = optimizers.Adamax(learning_rate = 0.001)
VSnet.compile(loss = loss, optimizer = optimizer)
 
VSnet.summary()

Training and saving the network

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1,
    mode='auto', baseline=None, restore_best_weights=True)
history = VSnet.fit(X, Y, validation_split=0.2, epochs=200, batch_size=32, verbose=1, callbacks = callback)
print(history.history.keys())

VSnet.save('Weights/01_SensorPlacementTest/DNNModel%s_IncWave%s.h5' % (modelID, include_wave)) # For sensor Placement

In [ ]:
cm = 1/2.54  # centimeters in inches
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 9
nn_output = VSnet.predict(X)

fig, ax = plt.subplots(1, figsize=(17*cm, 9*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.2, right=.98, top=0.98, bottom=0.18, hspace = 0.65, wspace=0.15)
ax.plot(Y[0:600,-1], linewidth=0.3)
ax.plot(nn_output[0:600,-1], linewidth=0.3)
fig, ax = plt.subplots(1, figsize=(17*cm, 9*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.2, right=.98, top=0.98, bottom=0.18, hspace = 0.65, wspace=0.15)
ax.plot(Y[0:600,-2], linewidth=0.3)
ax.plot(nn_output[0:600,-2], linewidth=0.3)

MAE = metrics.mean_absolute_error(Y, nn_output)
print(MAE)